In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dog-vs-cat-vs-bird/dataset/sample_submission.csv
/kaggle/input/dog-vs-cat-vs-bird/dataset/train.csv
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_1963.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_687.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_1603.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_421.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_1345.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_837.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_834.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_2384.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_782.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_1835.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_2703.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_2814.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_2409.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_1529.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_2688.png
/kaggl

In [10]:
import pandas as pd
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [15]:
import torch.nn as nn
import torch.optim as optim

In [55]:
from sklearn.preprocessing import LabelEncoder

In [48]:
train_csv = pd.read_csv('/kaggle/input/dog-vs-cat-vs-bird/dataset/train.csv')
print(train_csv['label'].unique())  # Ensure this outputs only [0, 1, 2]

['cat' 'dog' 'bird']


In [49]:
images, labels = next(iter(train_loader))
outputs = model(images.to(device))
print(f"Model output shape: {outputs.shape}")  # Should be [batch_size, 3]
print(f"Labels: {labels}")

Model output shape: torch.Size([64, 3])
Labels: ('cat', 'cat', 'cat', 'bird', 'bird', 'cat', 'cat', 'bird', 'bird', 'cat', 'cat', 'bird', 'cat', 'dog', 'bird', 'cat', 'cat', 'dog', 'bird', 'cat', 'bird', 'cat', 'bird', 'cat', 'cat', 'cat', 'bird', 'cat', 'dog', 'bird', 'cat', 'dog', 'bird', 'bird', 'bird', 'bird', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'bird', 'bird', 'bird', 'bird', 'dog', 'dog', 'cat', 'bird', 'dog', 'bird', 'cat', 'bird', 'bird', 'dog', 'bird', 'cat', 'cat', 'dog', 'cat', 'dog', 'bird')


In [77]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        # Read CSV and store the image data and labels
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
        # Initialize the LabelEncoder to convert categorical labels to numeric
        self.label_encoder = LabelEncoder()
        # Fit the encoder on all unique labels
        self.data.iloc[:, 1] = self.label_encoder.fit_transform(self.data.iloc[:, 1])

    def __len__(self):
        # Return the total number of samples
        return len(self.data)

    def __getitem__(self, idx):
        # Get the image path and label
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])  # Image filename
        image = Image.open(img_name).convert('RGB')  # Open and convert to RGB
        label = self.data.iloc[idx, 1]  # Categorical label (already encoded to integer)

        # Apply transformation if any (e.g., resize, normalize)
        if self.transform:
            image = self.transform(image)

        return image, label

    def get_label_encoder(self):
        # Return the fitted label encoder
        return self.label_encoder

In [78]:
img_path = '/kaggle/input/dog-vs-cat-vs-bird/dataset/train/cat_9146.png'
image = Image.open(img_path)
print(f"Original Image Mode: {image.mode}")  # Should show 'RGBA'

image = image.convert('RGB')
print(f"Converted Image Mode: {image.mode}")  # Should show 'RGB'

Original Image Mode: RGBA
Converted Image Mode: RGB


In [79]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [80]:
train_dataset = CustomDataset(
    csv_file='/kaggle/input/dog-vs-cat-vs-bird/dataset/train.csv',
    root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/train',
    transform=transform
)

In [84]:
label_encoder = train_dataset.get_label_encoder()

In [81]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [82]:
images, labels = next(iter(train_loader))
print(f"Image batch shape: {images.shape}")  # Should be [batch_size, 3, height, width]

Image batch shape: torch.Size([64, 3, 32, 32])


In [60]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [64]:
def train_model(model, train_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, correct = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss/len(train_loader):.4f}, Accuracy: {correct/len(train_dataset):.4f}")

In [65]:
train_model(model, train_loader, epochs=10)

Epoch 1/10 | Loss: 0.9269, Accuracy: 0.5503
Epoch 2/10 | Loss: 0.7895, Accuracy: 0.6435
Epoch 3/10 | Loss: 0.7213, Accuracy: 0.6809
Epoch 4/10 | Loss: 0.6542, Accuracy: 0.7212
Epoch 5/10 | Loss: 0.5996, Accuracy: 0.7479
Epoch 6/10 | Loss: 0.5413, Accuracy: 0.7727
Epoch 7/10 | Loss: 0.4826, Accuracy: 0.7993
Epoch 8/10 | Loss: 0.4106, Accuracy: 0.8364
Epoch 9/10 | Loss: 0.3485, Accuracy: 0.8643
Epoch 10/10 | Loss: 0.2749, Accuracy: 0.8960


In [69]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_names = sorted(os.listdir(root_dir))  # List of image filenames in the test directory
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_names[idx])  # Path to the image
        image = Image.open(img_name).convert('RGB')  # Convert to RGB to handle RGBA images

        if self.transform:
            image = self.transform(image)  # Apply transformations if any

        return image, self.image_names[idx]  # Return image and filename for tracking

In [70]:
test_dataset = TestDataset(root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [71]:
model.eval()
predictions = []
filenames = []

with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(1).cpu().numpy()  # Get class predictions (indices)
        predictions.extend(preds)
        filenames.extend(file_names)  # Keep track of filenames

In [85]:
predicted_labels = label_encoder.inverse_transform(predictions)

In [112]:
submission_df = pd.DataFrame({
    'filename': filenames,
    'label': predicted_labels
})

In [113]:
submission_df.head()

,filename,label
0,img_0.png,dog
1,img_1.png,dog
2,img_10.png,cat
3,img_100.png,cat
4,img_1000.png,cat


In [114]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

In [115]:
submission_df = pd.read_csv('/kaggle/working/submission.csv')
print(submission_df.head())

       filename label
0     img_0.png   dog
1     img_1.png   dog
2    img_10.png   cat
3   img_100.png   cat
4  img_1000.png   cat


In [116]:
sample_submission_df = pd.read_csv('/kaggle/input/dog-vs-cat-vs-bird/dataset/sample_submission.csv')
print(sample_submission_df.head())

    filename label
0  img_0.png   cat
1  img_1.png   dog
2  img_2.png  bird
3  img_3.png   cat
4  img_4.png   dog
